In [1]:
!md5sum customer.csv
!md5sum ../shd_t2/customer.csv

629b6682896bc985b595352090cb223d  customer.csv
629b6682896bc985b595352090cb223d  ../shd_t2/customer.csv


# Создание таблиц

In [12]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS customer_20240101 (
        customer_id INT PRIMARY KEY,
        first_name VARCHAR(50),
        last_name VARCHAR(50),
        gender VARCHAR(30),
        dob VARCHAR(50),
        job_title VARCHAR(50),
        job_industry_category VARCHAR(50),
        wealth_segment VARCHAR(50),
        deceased_indicator VARCHAR(50),
        owns_car VARCHAR(30),
        address VARCHAR(50),
        postcode VARCHAR(30),
        state VARCHAR(30),
        country VARCHAR(30),
        property_valuation INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS transaction_20240101 (
        transaction_id INT PRIMARY KEY,
        product_id INT,
        customer_id INT REFERENCES customer_20240101(customer_id) ON DELETE CASCADE,
        transaction_date VARCHAR(30),
        online_order VARCHAR(30),
        order_status VARCHAR(30),
        brand VARCHAR(30),
        product_line VARCHAR(30),
        product_class VARCHAR(30),
        product_size VARCHAR(30),
        list_price FLOAT,
        standard_cost FLOAT
    )
''')

conn.commit()

def load_csv_to_db(csv_path, table_name, skip_invalid=False):
    df = pd.read_csv(csv_path, sep=';', decimal=',')
    df.columns = [col.strip() for col in df.columns]
    if 'transaction_date' in df.columns:
        df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')

    if table_name == "transaction_20240101":
        cursor.execute("SELECT customer_id FROM customer_20240101")
        valid_customers = {row[0] for row in cursor.fetchall()}
        
        df = df[df["customer_id"].isin(valid_customers)]
    
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['%s'] * len(row))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values}) ON CONFLICT DO NOTHING"
        cursor.execute(insert_query, tuple(row))
    
    conn.commit()

load_csv_to_db('customer.csv', 'customer_20240101')
load_csv_to_db('transaction.csv', 'transaction_20240101')

cursor.close()
conn.close()


# Создание запросов

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

def execute_query(query, description):
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    print(f"\n{description}:")
    print(df)
    return df

query1 = '''
    SELECT job_industry_category, COUNT(*) AS customer_count
    FROM customer_20240101
    GROUP BY job_industry_category
    ORDER BY customer_count DESC;
'''
execute_query(query1, "Распределение клиентов по сферам деятельности")

query2 = '''
    SELECT DATE_TRUNC('month', t.transaction_date::DATE) AS month,
           c.job_industry_category,
           SUM(t.list_price) AS total_sales
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    GROUP BY month, c.job_industry_category
    ORDER BY month, c.job_industry_category;
'''
execute_query(query2, "Сумма транзакций по месяцам и сферам деятельности")

query3 = '''
    SELECT brand, COUNT(*) AS online_orders
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    WHERE t.online_order = 'True' AND t.order_status = 'Approved' AND c.job_industry_category = 'IT'
    GROUP BY brand;
'''
execute_query(query3, "Количество онлайн-заказов для брендов в сфере IT")

query4_1 = '''
    SELECT customer_id, SUM(list_price) AS total_spent,
           MAX(list_price) AS max_transaction,
           MIN(list_price) AS min_transaction,
           COUNT(*) AS transaction_count
    FROM transaction_20240101
    GROUP BY customer_id
    ORDER BY total_spent DESC, transaction_count DESC;
'''
execute_query(query4_1, "Сумма, макс., мин. и количество транзакций (GROUP BY)")

query4_2 = '''
    SELECT DISTINCT customer_id, 
           SUM(list_price) OVER (PARTITION BY customer_id) AS total_spent,
           MAX(list_price) OVER (PARTITION BY customer_id) AS max_transaction,
           MIN(list_price) OVER (PARTITION BY customer_id) AS min_transaction,
           COUNT(*) OVER (PARTITION BY customer_id) AS transaction_count
    FROM transaction_20240101
    ORDER BY total_spent DESC, transaction_count DESC;
'''
execute_query(query4_2, "Сумма, макс., мин. и количество транзакций (оконные функции)")

query5_min = '''
    SELECT first_name, last_name, SUM(list_price) AS total_spent
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    GROUP BY first_name, last_name
    ORDER BY total_spent ASC
    LIMIT 1;
'''
execute_query(query5_min, "Клиент с минимальной суммой транзакций")

query5_max = '''
    SELECT first_name, last_name, SUM(list_price) AS total_spent
    FROM transaction_20240101 t
    JOIN customer_20240101 c ON t.customer_id = c.customer_id
    GROUP BY first_name, last_name
    ORDER BY total_spent DESC
    LIMIT 1;
'''
execute_query(query5_max, "Клиент с максимальной суммой транзакций")

query6 = '''
    SELECT customer_id, transaction_id, transaction_date, list_price
    FROM (
        SELECT customer_id, transaction_id, transaction_date, list_price,
               ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY transaction_date ASC) AS rn
        FROM transaction_20240101
    ) sub
    WHERE rn = 1;
'''
execute_query(query6, "Самые первые транзакции клиентов")

query7 = '''
    WITH diff AS (
        SELECT c.customer_id, first_name, last_name, job_title,
               transaction_date::DATE - LAG(transaction_date::DATE) OVER (PARTITION BY c.customer_id ORDER BY transaction_date) AS diff_days
        FROM transaction_20240101 t
        JOIN customer_20240101 c ON t.customer_id = c.customer_id
    )
    SELECT first_name, last_name, job_title, MAX(diff_days) AS max_interval
    FROM diff
    WHERE diff_days IS NOT NULL
    GROUP BY first_name, last_name, job_title
    ORDER BY max_interval DESC
    LIMIT 1;
'''
execute_query(query7, "Клиенты с максимальным интервалом между транзакциями")

cursor.close()
conn.close()


Распределение клиентов по сферам деятельности:
  job_industry_category  customer_count
0         Manufacturing             799
1    Financial Services             774
2                   NaN             656
3                Health             602
4                Retail             358
5              Property             267
6                    IT             223
7         Entertainment             136
8           Argiculture             113
9    Telecommunications              72

Сумма транзакций по месяцам и сферам деятельности:
                        month job_industry_category  total_sales
0   2017-01-01 00:00:00+05:00           Argiculture     43513.82
1   2017-01-01 00:00:00+05:00         Entertainment     64089.92
2   2017-01-01 00:00:00+05:00    Financial Services    366383.71
3   2017-01-01 00:00:00+05:00                Health    286860.38
4   2017-01-01 00:00:00+05:00                    IT    107783.37
..                        ...                   ...          ...
115 2